<a href="https://colab.research.google.com/github/robert-lemon-uhm/NOAA_King_Tide/blob/main/NOAA_King_Tide_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before running, make sure the *Configuration Settings* below is correct.

To run this notebook and generate the Google Sheets file, click `Runtime -> Run All`.

## Configuration Settings

In [ ]:
#@title Data Range { run: "auto" }

#@markdown Please select a start and end date.
#@markdown This will be the range of king tide predictions.

start_date = '2021-01-01' #@param {type:"date"}
end_date = '2021-12-31' #@param {type:"date"}

In [ ]:
#@title Reasonable Times { run: "auto" }

#@markdown Please select an earliest and latest time for reasonable tide predictions.
#@markdown All tide predictions outside of this time range will be filtered out. 
#@markdown The earliest time cannot be later than the latest time.
#@markdown 
#@markdown The time is formatted in 24-hour (military) time as follows: &nbsp;&nbsp;`HH:MM`
#@markdown
#@markdown The default is `07:00` - `19:00` (7AM  - 7PM).

earliest_time = '07:00' #@param {type:"string"}
latest_time = '19:00' #@param {type:"string"}

In [ ]:
#@title Number of Predictions { run: "auto" }

#@markdown Please specify the number of predictions to be generated. 
#@markdown This is the amount of top predictions that will appear on each page for each station, 
#@markdown including the composite score table.
#@markdown
#@markdown The number of desired predictions cannot exceed the number of days between the start and end dates.

x_top_tides =  25#@param {type:"integer"}

In [ ]:
#@title Composite Score Settings { run: "auto" }

#@markdown Please select a geographical region to get tide predictions for.

region = 'Hawaii' #@param ['Northern Mariana Islands', 'Federated States of Micronesia', 'Marshall Islands', 'Hawaii', 'American Samoa']

#@markdown Please also enter the primary station ID to use for the composite score table.
#@markdown The default station ID is `1612340 (Honolulu)`. Station IDs can be found [here](https://tidesandcurrents.noaa.gov/tide_predictions.html?gid=1749#listing).

primary_station_ID =  1612340#@param {type:"integer"}

**NOTE:** If you double click on the forms above, it will display the code for them. If you accidentally do this, you can always hide the code by double clicking the form again. Alternatively, single clicking on the form, clicking the three vertical dots at the top right of the form, and selecting `Form` -> `Hide Code` will also hide the code.

## Source Code

#### Imports and Methods

In [ ]:
# Default python3 imports
import json
import operator
from urllib.request import urlopen
from datetime import datetime

# Third party data science imports (installed by default on Google Colab)
import numpy as np
import pandas as pd

# Third party google drive imports (installed by default on Google Colab)
from google.colab import auth
import gspread
from google.auth import default

In [ ]:
def get_station_name(station_ID):
  '''
  DESC:   A function that uses the NOAA metadata API to get the name of a tide prediction station,
          given its station ID. 
          API LINK: https://tidesandcurrents.noaa.gov/api-helper/url-generator.html

  INPUT:  'station_ID'    the ID of the station to get the name of

  OUTPUT: the name of the station
  '''
  API_response = urlopen(f"https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations/{station_ID}.json?units=english")
  station_json = json.loads(API_response.read())
  return station_json['stations'][0]['name']

In [ ]:
def create_URL(start_date, end_date, station_ID, time_zone='lst', units='english'):
  '''
  DESC:   A function to build the NOAA API URL for tide predictions.
          API LINK: https://tidesandcurrents.noaa.gov/api-helper/url-generator.html

  INPUT:  'start_date'    the start date for the data (format "YYYYMMDD")
          'end_date'      the end date for the data (format "YYYYMMDD")
          'station_ID'    the ID of the station to get tide data from
          'time_zone'     (OPTIONAL) the time zone
                              possible options: "gmt", "lst", "lst_ldt" (default is "lst")
          'units'         (OPTIONAL) the unit of measurement for tide prediction
                              possible options: "english", "metric" (default is "english")

  OUTPUT: the NOAA API URL containing the data in JSON format
  '''

  assert time_zone == 'lst' or time_zone == 'gmt' or time_zone == 'lst_ldt', \
         "ERROR: Incorrect input for 'time_zone' parameter!"
  assert units == 'english' or units == 'metric', \
         "ERROR: Incorrect input for 'units' parameter!"

  return "https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?" \
    + f"begin_date={start_date}&end_date={end_date}&station={station_ID}&" + \
    f"product=predictions&datum=MLLW&time_zone={time_zone}" + \
    f"&interval=hilo&units={units}&format=json"

In [ ]:
def download_API_data(url):
  '''
  DESC:   A function to download a .json file from a NOAA API URL.
          Data is stored in a dictionary {date : tide level}. Assumes 
          that a valid URL is passed in (use the 'create_URL' method).

  INPUT:  'url'   the NOAA API URL for the data download

  OUTPUT: the downloaded data, in a dictionary
  '''

  API_response = urlopen(url)
  json_data = json.loads(API_response.read())
  return {x['t']: float(x['v']) for x in json_data['predictions']}

In [ ]:
def remove_unreasonable_times(df):
  '''
  DESC:   A function to remove rows of a dataframe whose time is unreasonable.
          This assumes that the index column of the dataframe is a date STRING.

  INPUT:  'df'   The dataframe to filter

  OUTPUT: the inputted dataframe, with unreasonably timed rows removed
  '''

  # Get min and max times
  min_time = datetime.strptime(earliest_time, '%H:%M').time()
  max_time = datetime.strptime(latest_time, '%H:%M').time()
  # Remove unreasonable times from df
  return df[[min_time < datetime.strptime(time, '%Y-%m-%d %H:%M').time() < max_time 
                for time in df.index.values.tolist()]]

In [ ]:
def consolidate_dates(df):
  '''
  DESC:   A function that consolidates values by date. Only the max value for 
          each date is kept, all other rows with that date is dropped. The 
          index is then simplified to only include the date, not the time. It 
          is assumed that the index column of the dataframe is a date STRING, 
          called 'Date'. 

  INPUT:  'df'    The dataframe to considate

  OUTPUT: the inputted dataframe, with dates consolidated
  '''

  # Drop the time
  df['Date'] = [x.split()[0] for x in df.index]
  df = df.set_index('Date')
  # Keep only the max for each date
  return df.groupby('Date').max()

In [ ]:
def sort_and_rank(df, station_name):
  '''
  DESC:   A function that sorts a dataframe by its 'station_name' column.
          After sorting, it overwrites this column with the row's rank.

  INPUT:  'df'            The dataframe to sort and rank
          'station_name'  The name of the column to sort and rank

  OUTPUT: the df, sorted and ranked
  '''

  df = df.sort_values(station_name, ascending=False)
  df[station_name] = np.arange(1, df.shape[0]+1, dtype=int)
  return df

#### Driver Code

Processing of start/end date Configuration Settings

In [ ]:
# Reformat start and end dates
start_date = datetime.strptime(start_date, "%Y-%m-%d").strftime("%Y%m%d")
end_date = datetime.strptime(end_date, "%Y-%m-%d").strftime("%Y%m%d")

# Check the validity of the requested number of predictions
num_of_days = (datetime.strptime(end_date, "%Y%m%d") - datetime.strptime(start_date, "%Y%m%d")).days
assert num_of_days >= x_top_tides
assert num_of_days > 1

# Check the validity of the reasonable time frame
assert datetime.strptime(earliest_time, "%H:%M") < datetime.strptime(latest_time, "%H:%M")

Processing of station/region Configuration Settings

In [ ]:
# Create lists of all harmonic stations for each region
NMI_station_IDs = [1633227, 1630000, 1631428]
FSM_station_IDs = [1840000]
MI_station_IDs = [1890000, 1820000]
H_station_IDs = [1611347, 1611400, 1612366, 1612404, 1612340, 1612480, 1613198, 1615680, 1617433, 1617760]
AS_station_IDs = [1770000]

# Determine which region was selected
if region == 'Northern Mariana Islands':
  station_IDs = NMI_station_IDs.copy()
elif region == 'Federated States of Micronesia':
  station_IDs = FSM_station_IDs.copy()
elif region == 'Marshall Islands':
  station_IDs = MI_station_IDs.copy()
elif region == 'American Samoa':
  station_IDs = AS_station_IDs.copy()
else: # default is 'Hawaii'
  station_IDs = H_station_IDs.copy()

# Build stations dict
stations = {get_station_name(station_ID): station_ID for station_ID in station_IDs}

# Determine if multiple stations have been selected (including primary station)
has_multiple = True
if len(stations.keys()) == 1 and primary_station_ID in stations.values():
  has_multiple = False

Initial setup for creating the google sheets file

In [ ]:
# Connect to google drive
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Create new sheets doc
start_date_formatted = datetime.strptime(start_date, "%Y%m%d").strftime("%m.%d.%Y")
end_date_formatted = datetime.strptime(end_date, "%Y%m%d").strftime("%m.%d.%Y")
sh = gc.create(f"{region} [{start_date_formatted} - {end_date_formatted}]")

Download, data wrangling, and google sheets upload of primary station used in composite table (probably Honolulu).

In [ ]:
# Get primary station name
primary_station_name = ''
if primary_station_ID in stations.values():
  primary_station_name = list(stations.keys())[list(stations.values()).index(primary_station_ID)]
else:
  primary_station_name = get_station_name(primary_station_ID)

# Get primary station data into df
API_URL = create_URL(start_date, end_date, primary_station_ID)
dict_data = download_API_data(API_URL)
composite_df = pd.DataFrame.from_dict(dict_data, orient='index', columns=[primary_station_name])
composite_df = remove_unreasonable_times(composite_df)

# Data wrangling to get google drive df
primary_station_df = composite_df.copy()
primary_station_df = primary_station_df.sort_values(primary_station_name, ascending=False)
primary_station_df = primary_station_df.iloc[:x_top_tides, :]

# Data wrangling to get final composite df
composite_df = consolidate_dates(composite_df)
composite_df = sort_and_rank(composite_df, primary_station_name)

# Create new sheet for primary station and write to it
sh.add_worksheet(title=primary_station_name, rows=primary_station_df.shape[0], cols=3)
worksheet = sh.worksheet(primary_station_name)
worksheet.update('A1', [['Date']] + [[x.split()[0]] for x in primary_station_df.index.values])
worksheet.update('B1', [['Time']] + [[x.split()[1]] for x in primary_station_df.index.values])
worksheet.update('C1', ([primary_station_df.columns.values.tolist()] + primary_station_df.values.tolist()))

{'spreadsheetId': '1CYcDLFqS5TP15u6vjgJBt-WS1oy-R4oegCeNrPA2eGI',
 'updatedCells': 26,
 'updatedColumns': 1,
 'updatedRange': 'Honolulu!C1:C26',
 'updatedRows': 26}

Download, data wrangling, and google sheets upload of all non-primary stations used in composite table.

In [ ]:
if has_multiple:
  # Add each station's data to the composite df
  for station in stations:

    station_ID = stations[station]

    if station_ID != primary_station_ID:
      # Get station data into df
      API_URL = create_URL(start_date, end_date, station_ID)
      dict_data = download_API_data(API_URL)
      station_df_composite = pd.DataFrame.from_dict(dict_data, orient='index', columns=[station])
      station_df_composite = remove_unreasonable_times(station_df_composite)

      # Data wrangling to get google sheets df
      station_df_sheets = station_df_composite.copy()
      station_df_sheets = station_df_sheets.sort_values(station, ascending=False)
      station_df_sheets = station_df_sheets[:x_top_tides]

      # Add google sheets df to google sheets
      sh.add_worksheet(title=station, rows=station_df_sheets.shape[0], cols=3)
      worksheet = sh.worksheet(station)
      worksheet.update('A1', [['Date']] + [[x.split()[0]] for x in station_df_sheets.index.values])
      worksheet.update('B1', [['Time']] + [[x.split()[1]] for x in station_df_sheets.index.values])
      worksheet.update('C1', ([station_df_sheets.columns.values.tolist()] + station_df_sheets.values.tolist()))

      # Data wrangling to get composite df
      station_df_composite = consolidate_dates(station_df_composite)
      station_df_composite = sort_and_rank(station_df_composite, station)

      # Add station df to composite df
      composite_df = composite_df.join(station_df_composite[station])

  # Compute composite score
  composite_df['Composite Score'] = composite_df.sum(axis=1)
  composite_df = composite_df[:x_top_tides]

  # Replace invalid dates with -1
  rows_with_na = np.array(composite_df.isna().sum(axis=1))
  composite_df.loc[rows_with_na > 0, 'Composite Score'] = -1
  composite_df = composite_df.fillna(-1)
  composite_df = composite_df.astype(int)

  # Add composite table to google sheets
  worksheet = sh.sheet1
  worksheet.update_title("COMPOSITE SCORE")
  worksheet.update('A1', [['Date']] + [[x] for x in composite_df.index.values])
  worksheet.update('B1', [composite_df.columns.values.tolist()] + composite_df.values.tolist())
  worksheet.update(f"A{x_top_tides+3}", "NOTE: Composite scores of -1 indicate that one of the stations don't have a reasonably timed high tide for that day")

# Custom composite page if only one station
else:
  worksheet = sh.sheet1
  worksheet.update_title("COMPOSITE SCORE")
  worksheet.update('A1', f"Only one harmonic station selected for [{region}] region, so there is no composite table")

In [ ]:
composite_df.head(10)

,Honolulu,"PORT ALLEN, HANAPEPE BAY, KAUAI ISLAND",Nawiliwili,"FORT KAMEHAMEHA, BISHOP POINT, PEARL HBR","FORD ISLAND, FERRY DOCK, PEARL HARBOR",Mokuoloe,Kaunakakai Harbor,"Kahului, Kahului Harbor",Kawaihae,"Hilo, Hilo Bay, Kuhio Bay",Composite Score
Date,,,,,,,,,,,
2021-06-24,1,1,1,1,1,1,1,2,1,1,11
2021-07-22,2,2,2,2,2,3,4,3,4,4,28
2021-07-23,3,3,3,4,3,2,5,1,2,2,28
2021-06-23,4,4,5,3,4,5,2,6,5,5,43
2021-06-25,5,5,4,6,5,4,3,4,3,3,42
2021-07-21,6,6,6,5,6,7,9,13,9,12,79
2021-07-24,7,7,7,9,7,6,8,5,6,6,68
2021-05-26,8,10,11,7,8,10,6,10,8,7,85
2021-05-27,9,11,10,10,12,11,7,11,7,8,96


##### Developer Note:

The code in the cell to get the primary station data previously sometimes generated a warning, but I believe it to be fixed now. I was using 

`primary_station_df = primary_station_df[:x_top_tides]` 

to drop everything but the first `x_top_tides` rows from the dataframe, but have since switched it to 

`primary_station_df = primary_station_df.iloc[:x_top_tides, :]`

which is the recommended way, and appears to no longer generate the warning. The warning that would occassionally generate was as follows:

> /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

> See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app